In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,learning_curve
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error,mean_squared_log_error
from sklearn.preprocessing import LabelEncoder
import xgboost
import matplotlib.pyplot as plt
import seaborn as sns
from yellowbrick.model_selection import ValidationCurve
from yellowbrick.regressor import PredictionError
import pickle

In [3]:
data = pd.read_csv('/content/NW_Ground_Stations_2016.csv')

In [4]:
Corr = df.corr()
sns.heatmap(Corr, xticklabels=Corr.columns, yticklabels=Corr.columns, annot=True,cmap="YlGnBu")

In [ ]:
df.info()

In [ ]:
df.describe()

In [8]:
df.head()

,Unnamed: 0,t,date,number_sta
0,0,279.85,20160101 00:00,0
1,1,278.45,20160101 00:00,1
2,2,277.65,20160101 00:00,2
3,3,278.95,20160101 00:00,3
4,4,278.35,20160101 00:00,4


In [9]:
df = data[['t','date','number_sta']]

In [10]:
df['t'] = df['t'].interpolate(method ='linear', limit_direction ='forward')

In [11]:
df.head()

,t,date,number_sta
0,279.85,20160101 00:00,0
1,278.45,20160101 00:00,1
2,277.65,20160101 00:00,2
3,278.95,20160101 00:00,3
4,278.35,20160101 00:00,4


In [12]:
df['number_sta']= LabelEncoder().fit_transform(df['number_sta'])

In [13]:
df['date'] = pd.to_datetime(df['date'], format='%Y%m%d %H:%M')
df = df.set_index('date')

df['Year']= df.index.year
df['Month'] = df.index.month
df['Day'] = df.index.day
df['DW'] = df.index.dayofweek
df['DY'] = df.index.dayofyear
df['WY'] = df.index.weekofyear

df['H'] = df.index.hour
df['M'] = df.index.minute
df['S'] = df.index.second

In [14]:
df.head()

,t,number_sta,Year,Month,Day,DW,DY,WY,H,M,S
date,,,,,,,,,,,
2016-01-01,279.85,0,2016,1,1,4,1,53,0,0,0
2016-01-01,278.45,1,2016,1,1,4,1,53,0,0,0
2016-01-01,277.65,2,2016,1,1,4,1,53,0,0,0
2016-01-01,278.95,3,2016,1,1,4,1,53,0,0,0
2016-01-01,278.35,4,2016,1,1,4,1,53,0,0,0


In [15]:
X = df.drop(columns=['t'],axis=1)
Y = df['t']

In [16]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)


In [17]:
# # GBR = GradientBoostingRegressor()
# # GBR.fit(X_train,Y_train)
# # pred = GBR.predict(X_test)

# Result >> r2= 0.699405129313601 MAE= 2.500495975739092 MSE= 10.607390609037203

In [18]:
XGB = xgboost.XGBRegressor()

In [ ]:
model = XGB
visualizer = PredictionError(model)

visualizer.fit(X_train, Y_train)  
visualizer.score(X_test, Y_test)  
visualizer.show()  

In [ ]:
viz = ValidationCurve(
    XGB, param_name="max_depth",
    param_range=np.arange(1, 11), cv=4, scoring="r2"
)

# Fit and show the visualizer
viz.fit(X, Y)
viz.show()

In [ ]:
XGB.fit(X_train,Y_train)

In [ ]:
plt.bar( range(len(XGB.feature_importances_)), XGB.feature_importances_)
plt.xticks(range(len(XGB.feature_importances_)), df.columns)
plt.show()

In [ ]:
pred = XGB.predict(X_test)


In [ ]:
print('r2=',r2_score(Y_test,pred),'MAE=',mean_absolute_error(Y_test,pred),'MSE=',mean_squared_error(Y_test,pred))



# # #Result >> r2_score= 0.9276552674827134 MAE= 1.3399149728369535 MSE= 3.0532917890705478

In [ ]:
df.to_csv('NW_Ground_Stations_2016final3.csv')

pickle.dump(XGB,open('weather.sav','wb'))